In [1]:
import telegram

from telegram import (
    Poll,
    ParseMode,
    KeyboardButton,
    KeyboardButtonPollType,
    ReplyKeyboardMarkup,
    ReplyKeyboardRemove,
    InlineKeyboardButton,
    InlineKeyboardMarkup,
#     ReplyKeyboardMarkup
    Update,
    Bot,
)
# from telegram.util import (

# build_menu
# )

from telegram.ext import (
    Updater,
    CommandHandler,
    PollAnswerHandler,
    PollHandler,
    MessageHandler,
    Filters,
    CallbackContext,
    Handler,
    CallbackQueryHandler,
    PollHandler,
)

import logging

logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO,
)


# array_o

In [2]:
import numpy as np
import sklearn
import pandas as pd
from scipy.spatial.distance import cosine

In [3]:
survey_dict = {
    
#     'q0' : [('WHAT WILL YOU EAT', ),
#             (
#                 "MAIN DISH",
#                 "DAIRY",
#                 "VEGETABLE",
#                 "SWEET"
#             )
        
#     ],
    'q1': [
        ("MEAT",),
        (
            "Red meat",
            "Cured meat ",
            "Pork",
            "Poiltry",
            "Mollusk",
            "Fish",
            "Lobster & Shellfish",
#             "SKIP THIS ONE"
        ),
    ],
    
    'q2': [
        ("Preparation",),
        (
            "Grilled or Barbecued",
            "Sauteed or Fried",
            "Smoked",
            "Roasted",
            "Poached or steamed",
#             "SKIP THIS ONE"
        ),
    ],
    
    'q3': [
        ("DAIRY",),
        (
            "Soft cheese and cream", 
            "Pungent cheese", 
            "Hard Cheese", 
            "SKIP THIS ONE"
          
        ),
    ],
    
    'q4': [
        ("VEGETABLE",),
        (
            "Alliums", 
            "Green Vegetables", 
            "Root vegetable and squash",
            "nightshades",
            "Fungshi",
            "Nuts and seeds",
            "Beans and seeds",
            "SKIP THIS ONE"
        ),
    ],
    
    'q5': [
        ("Herbs an spices", ),
        (
            "Black pepper",
            "Red pepper",
            "Hot and spicy",
            "Herbs",
            "Baking spices",
            "Exotic aromatic spices",
            "SKIP THIS ONE"
        ),
    ],
    
    'q6': [
        ('Stratch',),
        (
           " White scratches",
           " Whole wheat grains",
           " Sweet scratchy vegetables",
           " Potato",
            "SKIP THIS ONE"
#            " I drink varied cola and fruit flavored soft drinks",
#            " I do not generally drink soft drinks",
        ),
    ],
    
    'q7' : [
            ('Sweet:',),
            (
                 "Fruit and berries",
                 "Vanilla and caramel",
                 "Chocolate and coffee",
                "SKIP THIS ONE"
#                  "Crystal Light low calorie lemonade",
#                  "Do not care for lemonade",
            ),
    ]

}

# arr_q = ["q1", "q2", "q3", "q4", "q5","q6", "q7", "q8", "q9" ]

In [4]:
# Stages
FIRST, SECOND, THIRD , FOURTH, FIFTH, SIXTH, SEVENTH, EIGHTS= range(8)
# Callback data
ONE, TWO, THREE, FOUR = range(4)

In [5]:
def build_menu(buttons,
               n_cols,	
               header_buttons=None,
               footer_buttons=None):
	menu = [buttons[i:i + n_cols] for i in range(0, len(buttons), n_cols)]
	if header_buttons:
		menu.insert(0, [header_buttons])
	if footer_buttons:
		menu.append([footer_buttons])
	return menu

In [6]:
import logging

from telegram import InlineKeyboardButton, InlineKeyboardMarkup, Update
from telegram.ext import Updater, CommandHandler, CallbackQueryHandler, CallbackContext
import logging
from telegram import InlineKeyboardButton, InlineKeyboardMarkup, Update
from telegram.ext import (
    Updater,
    CommandHandler,
    CallbackQueryHandler,
    ConversationHandler,
    CallbackContext,
)

logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)
logger = logging.getLogger(__name__)


def start(update: Update, context: CallbackContext) -> None:
    arr_q = list(survey_dict.keys()) 
    array_of_answers = [0] * len(arr_q)
    """Sends a message with three inline buttons attached."""
    keyboard = []
    for i, el in enumerate(survey_dict['q1'][1]):
        print(el)
#         if i not in [0, 2, 4, 5]:
#             keyboard.append(InlineKeyboardButton(str(el), callback_data=str(100)))
#         else:
        keyboard.append(InlineKeyboardButton(str(el), callback_data=str(i)))
#     print(i, el )
#     keyboard = [
#         [
#             InlineKeyboardButton("Option 1", callback_data='1'),
#             InlineKeyboardButton("Option 2", callback_data='2'),
#         ],
#         [InlineKeyboardButton("Option 3", callback_data='3')],
#     ]

    reply_markup = InlineKeyboardMarkup(build_menu(keyboard,n_cols=1))

    message = update.message.reply_text(text = survey_dict['q1'][0][0], reply_markup=reply_markup)
    payload = {
        message.chat.id: {
            "questions": survey_dict['q1'][1],
            "message_id": message.message_id,
            "chat_id": update.effective_chat.id,
            "answers": 0,
            "count": 1,
            "arr_q": arr_q,
            "array_of_answers": array_of_answers,
        }
    }
    context.bot_data.update(payload)
    print(payload)
    return FIRST


def button_REDMEAT(update: Update, context: CallbackContext) -> None:
    
    query = update.callback_query
    chat_id = query.message.chat.id
    print(query)
    num_calls = context.bot_data[chat_id]['count']
    print(f"\nNUM CALLS!!!:{num_calls}\n")
    query_data = int(query.data)
    if num_calls < 7:
        context.bot_data[chat_id]['array_of_answers'][num_calls - 1] = query_data
    print(context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls-1]][1][query_data])
    
    keyboard = []
    for i, el in enumerate(np.array(context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls]][1])[[0,2,3]]):
#         print(el)
        keyboard.append(InlineKeyboardButton(str(el), callback_data=str(i)))
    reply_markup = InlineKeyboardMarkup(build_menu(keyboard,n_cols=1))
    message = query.edit_message_text(
        text=context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls]][0][0], reply_markup=reply_markup
    )
    payload = {
                chat_id: {
                    "questions": survey_dict['q1'][1],#questions,
                    "message_id": message.message_id,
                    "chat_id": context.bot_data[chat_id]["chat_id"],
                    "answers": 0,
                    "count": num_calls + 1,
                    "arr_q": context.bot_data[chat_id]["arr_q"],
                    "array_of_answers": context.bot_data[chat_id]['array_of_answers'],
                }
            }
    context.bot_data.update(payload)
            
    
    
    print(payload)
    return SECOND

def button_PORK(update: Update, context: CallbackContext) -> None:
    query = update.callback_query
    chat_id = query.message.chat.id
    print(query)
    num_calls = context.bot_data[chat_id]['count']
    print(f"\nNUM CALLS!!!:{num_calls}\n")
    query_data =int(query.data)
    if num_calls < 7:
        context.bot_data[chat_id]['array_of_answers'][num_calls - 1] = query_data
    print(context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls-1]][1][query_data])
    
    keyboard = []
    for i, el in enumerate(np.array(context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls]][1])[[0,1,2,3]]):
#         print(el)
        keyboard.append(InlineKeyboardButton(str(el), callback_data=str(i)))
    reply_markup = InlineKeyboardMarkup(build_menu(keyboard,n_cols=1))
    message = query.edit_message_text(
        text=context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls]][0][0], reply_markup=reply_markup
    )
    payload = {
                chat_id: {
                    "questions": survey_dict['q1'][1],#questions,
                    "message_id": message.message_id,
                    "chat_id": context.bot_data[chat_id]["chat_id"],
                    "answers": 0,
                    "count": num_calls + 1,
                    "arr_q": context.bot_data[chat_id]["arr_q"],
                    "array_of_answers": context.bot_data[chat_id]['array_of_answers'],
                }
            }
    context.bot_data.update(payload)
            
    
    
#     print(payload)
    return SECOND


def button_MOLLUSK(update: Update, context: CallbackContext) -> None:
    query = update.callback_query
    chat_id = query.message.chat.id
    print(query)
    num_calls = context.bot_data[chat_id]['count']
    print(f"\nNUM CALLS!!!:{num_calls}\n")
    query_data = int(query.data)
    if num_calls < 7:
        context.bot_data[chat_id]['array_of_answers'][num_calls - 1] = query_data
    print(context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls-1]][1][query_data])
    
    keyboard = []
    for i, el in enumerate(np.array(context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls]][1])[[0, 1,2]]):
#         print(el)
        keyboard.append(InlineKeyboardButton(str(el), callback_data=str(i)))
    reply_markup = InlineKeyboardMarkup(build_menu(keyboard,n_cols=1))
    message = query.edit_message_text(
        text=context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls]][0][0], reply_markup=reply_markup
    )
    payload = {
                chat_id: {
                    "questions": survey_dict['q1'][1],#questions,
                    "message_id": message.message_id,
                    "chat_id": context.bot_data[chat_id]["chat_id"],
                    "answers": 0,
                    "count": num_calls + 1,
                    "arr_q": context.bot_data[chat_id]["arr_q"],
                    "array_of_answers": context.bot_data[chat_id]['array_of_answers'],
                }
            }
    context.bot_data.update(payload)
            
    
    
#     print(payload)
    return SECOND


def button_FISH(update: Update, context: CallbackContext) -> None:
    query = update.callback_query
    chat_id = query.message.chat.id
    print(query)
    num_calls = context.bot_data[chat_id]['count']
    print(f"\nNUM CALLS!!!:{num_calls}\n")
    query_data=int(query.data)
    if num_calls < 7:
        context.bot_data[chat_id]['array_of_answers'][num_calls - 1] = query_data
    print(context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls-1]][1][query_data])
    
    keyboard = []
    for i, el in enumerate(np.array(context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls]][1])[[0,1,2,4]]):
#         print(el)
        keyboard.append(InlineKeyboardButton(str(el), callback_data=str(i)))
    reply_markup = InlineKeyboardMarkup(build_menu(keyboard,n_cols=1))
    message = query.edit_message_text(
        text=context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls]][0][0], reply_markup=reply_markup
    )
    payload = {
                chat_id: {
                    "questions": survey_dict['q1'][1],#questions,
                    "message_id": message.message_id,
                    "chat_id": context.bot_data[chat_id]["chat_id"],
                    "answers": 0,
                    "count": num_calls + 1,
                    "arr_q": context.bot_data[chat_id]["arr_q"],
                    "array_of_answers": context.bot_data[chat_id]['array_of_answers'],
                }
            }
    context.bot_data.update(payload)
            
    
    
#     print(payload)
    return SECOND

def button_OTHERS(update: Update, context: CallbackContext) -> None:
    query = update.callback_query
    chat_id = query.message.chat.id
    print(query)
    num_calls = context.bot_data[chat_id]['count']
    print(f"\nNUM CALLS!!!:{num_calls}\n")
    query_data=int(query.data)
    if num_calls < 7:
        context.bot_data[chat_id]['array_of_answers'][num_calls - 1] = query_data
    print(context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls-1]][1][query_data])
    
    keyboard = []
    for i, el in enumerate(np.array(context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls]][1])):
        print(el)
        keyboard.append(InlineKeyboardButton(str(el), callback_data=str(i)))
    reply_markup = InlineKeyboardMarkup(build_menu(keyboard,n_cols=1))
    message = query.edit_message_text(
        text=context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls]][0][0], reply_markup=reply_markup
    )
    payload = {
                chat_id: {
                    "questions": survey_dict['q1'][1],#questions,
                    "message_id": message.message_id,
                    "chat_id": context.bot_data[chat_id]["chat_id"],
                    "answers": 0,
                    "count": num_calls + 1,
                    "arr_q": context.bot_data[chat_id]["arr_q"],
                    "array_of_answers": context.bot_data[chat_id]['array_of_answers'],
                }
            }
    context.bot_data.update(payload)
            
    
    
    print(payload)
    return SECOND

def button(update: Update, context: CallbackContext) -> None:
    """Parses the CallbackQuery and updates the message text."""
    query = update.callback_query
    chat_id = query.message.chat.id
    print(query)
    num_calls = context.bot_data[chat_id]['count']
    print(f"\nNUM CALLS!!!:{num_calls}\n")
    if num_calls < 7:
        context.bot_data[chat_id]['array_of_answers'][num_calls - 1] = query.data
    print('BUTTON FIRST STARTS', query.data)
#     print(answer.option_ids)
#     answer = update.poll_answer
#     print(answer)
#     poll_id = answer.poll_id

    # CallbackQueries need to be answered, even if no notification to the user is needed
    # Some clients may have trouble otherwise. See https://core.telegram.org/bots/api#callbackquery
    query.answer()
    print('BUTTON FIRST STARTS', query.data)
    keyboard = [
        [
            InlineKeyboardButton("skfjk2", callback_data=str(4)),
            InlineKeyboardButton("mr3", callback_data=str(5)),
        ]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    query.edit_message_text(
        text="BUTTON", reply_markup=reply_markup
    )

#     query.edit_message_text(text=f"Selected option: {query.data}")
    

    print('BUTTON FIRST finish',query.data)
    return SECOND


def button_DAIRY(update: Update, context: CallbackContext) -> int:
    query = update.callback_query
    chat_id = query.message.chat.id
    print(query)
    num_calls = context.bot_data[chat_id]['count']
    print(f"\nNUM CALLS!!!:{num_calls}\n")
    query_data=int(query.data)
    if num_calls < 7:
        context.bot_data[chat_id]['array_of_answers'][num_calls - 1] = query_data
    print(context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls-1]][1][query_data])
    
    keyboard = []
    for i, el in enumerate(np.array(context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls]][1])):
        print(el)
        keyboard.append(InlineKeyboardButton(str(el), callback_data=str(i)))
    reply_markup = InlineKeyboardMarkup(build_menu(keyboard,n_cols=1))
    message = query.edit_message_text(
        text=context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls]][0][0], reply_markup=reply_markup
    )
    payload = {
                chat_id: {
                    "questions": survey_dict['q1'][1],#questions,
                    "message_id": message.message_id,
                    "chat_id": context.bot_data[chat_id]["chat_id"],
                    "answers": 0,
                    "count": num_calls + 1,
                    "arr_q": context.bot_data[chat_id]["arr_q"],
                    "array_of_answers": context.bot_data[chat_id]['array_of_answers'],
                }
            }
    context.bot_data.update(payload)
            
    
    
    print(payload)
    return THIRD

def button_VEGETABLE(update: Update, context: CallbackContext) -> int:
    query = update.callback_query
    chat_id = query.message.chat.id
    print(query)
    num_calls = context.bot_data[chat_id]['count']
    print(f"\nNUM CALLS!!!:{num_calls}\n")
    query_data=int(query.data)
    if num_calls < 7:
        context.bot_data[chat_id]['array_of_answers'][num_calls - 1] = query_data
    print(context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls-1]][1][query_data])
    
    keyboard = []
    for i, el in enumerate(np.array(context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls]][1])):
        print(el)
        keyboard.append(InlineKeyboardButton(str(el), callback_data=str(i)))
    reply_markup = InlineKeyboardMarkup(build_menu(keyboard,n_cols=1))
    message = query.edit_message_text(
        text=context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls]][0][0], reply_markup=reply_markup
    )
    payload = {
                chat_id: {
                    "questions": survey_dict['q1'][1],#questions,
                    "message_id": message.message_id,
                    "chat_id": context.bot_data[chat_id]["chat_id"],
                    "answers": 0,
                    "count": num_calls + 1,
                    "arr_q": context.bot_data[chat_id]["arr_q"],
                    "array_of_answers": context.bot_data[chat_id]['array_of_answers'],
                }
            }
    context.bot_data.update(payload)
            
    
    
    print(payload)
    return FOURTH

def button_HERBS(update: Update, context: CallbackContext) -> int:
    query = update.callback_query
    chat_id = query.message.chat.id
    print(query)
    num_calls = context.bot_data[chat_id]['count']
    print(f"\nNUM CALLS!!!:{num_calls}\n")
    query_data=int(query.data)
    if num_calls < 7:
        context.bot_data[chat_id]['array_of_answers'][num_calls - 1] = query_data
    print(context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls-1]][1][query_data])
    
    keyboard = []
    for i, el in enumerate(np.array(context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls]][1])):
        print(el)
        keyboard.append(InlineKeyboardButton(str(el), callback_data=str(i)))
    reply_markup = InlineKeyboardMarkup(build_menu(keyboard,n_cols=1))
    message = query.edit_message_text(
        text=context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls]][0][0], reply_markup=reply_markup
    )
    payload = {
                chat_id: {
                    "questions": survey_dict['q1'][1],#questions,
                    "message_id": message.message_id,
                    "chat_id": context.bot_data[chat_id]["chat_id"],
                    "answers": 0,
                    "count": num_calls + 1,
                    "arr_q": context.bot_data[chat_id]["arr_q"],
                    "array_of_answers": context.bot_data[chat_id]['array_of_answers'],
                }
            }
    context.bot_data.update(payload)
            
    
    
    print(payload)
    return FIFTH


def button_STRATCH(update: Update, context: CallbackContext) -> int:
    query = update.callback_query
    chat_id = query.message.chat.id
    print(query)
    num_calls = context.bot_data[chat_id]['count']
    print(f"\nNUM CALLS!!!:{num_calls}\n")
    query_data=int(query.data)
    if num_calls < 7:
        context.bot_data[chat_id]['array_of_answers'][num_calls - 1] = query_data
    print(context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls-1]][1][query_data])
    
    keyboard = []
    for i, el in enumerate(np.array(context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls]][1])):
        print(el)
        keyboard.append(InlineKeyboardButton(str(el), callback_data=str(i)))
    reply_markup = InlineKeyboardMarkup(build_menu(keyboard,n_cols=1))
    message = query.edit_message_text(
        text=context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls]][0][0], reply_markup=reply_markup
    )
    payload = {
                chat_id: {
                    "questions": survey_dict['q1'][1],#questions,
                    "message_id": message.message_id,
                    "chat_id": context.bot_data[chat_id]["chat_id"],
                    "answers": 0,
                    "count": num_calls + 1,
                    "arr_q": context.bot_data[chat_id]["arr_q"],
                    "array_of_answers": context.bot_data[chat_id]['array_of_answers'],
                }
            }
    context.bot_data.update(payload)
            
    
    
    print(payload)
    return SIXTH

def button_SWEET(update: Update, context: CallbackContext) -> int:
    query = update.callback_query
    chat_id = query.message.chat.id
    print(query)
    num_calls = context.bot_data[chat_id]['count']
    print(f"\nNUM CALLS!!!:{num_calls}\n")
    query_data=int(query.data)
    if num_calls < 7:
        context.bot_data[chat_id]['array_of_answers'][num_calls - 1] = query_data
    print(context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls-1]][1][query_data])
    
    keyboard = []
    for i, el in enumerate(np.array(context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls]][1])):
        print(el)
        keyboard.append(InlineKeyboardButton(str(el), callback_data=str(i)))
    reply_markup = InlineKeyboardMarkup(build_menu(keyboard,n_cols=1))
    message = query.edit_message_text(
        text=context.bot_data[context.bot_data[chat_id]['arr_q'][num_calls]][0][0], reply_markup=reply_markup
    )
    payload = {
                chat_id: {
                    "questions": survey_dict['q1'][1],#questions,
                    "message_id": message.message_id,
                    "chat_id": context.bot_data[chat_id]["chat_id"],
                    "answers": 0,
                    "count": num_calls + 1,
                    "arr_q": context.bot_data[chat_id]["arr_q"],
                    "array_of_answers": context.bot_data[chat_id]['array_of_answers'],
                }
            }
    context.bot_data.update(payload)
            
    
    
    print(payload)
    return SEVENTH

def button2(update: Update, context: CallbackContext) -> int:
    """Show new choice of buttons"""
    query = update.callback_query
    query.answer()
    print('BUTTON second STARTS', query.data)
    keyboard = [
        [
            InlineKeyboardButton("2", callback_data=str(6)),
            InlineKeyboardButton("3", callback_data=str(7)),
        ]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    query.edit_message_text(
        text="BUTTON 2", reply_markup=reply_markup
    )
    print('BUTTON second FINISHS', query.data)
    
    
    
    return THIRD

def end(update: Update, context: CallbackContext) -> int:
    """Returns `ConversationHandler.END`, which tells the
    ConversationHandler that the conversation is over.
    """
    query = update.callback_query
    chat_id = query.message.chat.id
    num_calls = context.bot_data[chat_id]['count']
    print(num_calls)
    query_data=int(query.data)
#     if num_calls < 7:
    context.bot_data[chat_id]['array_of_answers'][num_calls - 1] = query_data
#      query.edit_message_text(text=f"Selected option: {query.data}")
    
    query.answer()
    
    array_of_answers = context.bot_data[chat_id]['array_of_answers']
    text = get_wine_type(array_of_answers)
    text_len = len(text)
#         print(text_len)
    res = ' '.join([f"🍾 {text[i]}\n" for i in range(1,text_len)])
    res2 = text[0]+ ':' + '\n\n' +res
    query.edit_message_text(text=res2)
#     query.edit_message_text(text=f"See you next time!: {query.data}")
    return ConversationHandler.END
    

def help_command(update: Update, context: CallbackContext) -> None:
    """Displays info on how to use the bot."""
    update.message.reply_text("Use /start to test this bot.")


def main() -> None:
    """Run the bot."""
    # Create the Updater and pass it your bot's token.
    TOKEN = '2010745645:AAH6cEUziXN4F7Xh1geJwN64UspoQb_WZW0'
    updater = Updater(TOKEN)
    conv_handler = ConversationHandler(
            entry_points=[CommandHandler('start', start)],
            states={
                FIRST: [
                    CallbackQueryHandler(button_REDMEAT, pattern ='0'),
                    CallbackQueryHandler(button_PORK, pattern ='2'),
                    CallbackQueryHandler(button_MOLLUSK, pattern ='4'),
                    CallbackQueryHandler(button_FISH, pattern ='5'),
                    CallbackQueryHandler(button_OTHERS, pattern = '1'),
                    CallbackQueryHandler(button_OTHERS, pattern = '3'),
                    CallbackQueryHandler(button_OTHERS, pattern = '6')

                ],
                SECOND: [
                    CallbackQueryHandler(button_DAIRY),

                ],
                THIRD : [ 
                    CallbackQueryHandler(button_VEGETABLE)
                ],
                FOURTH : [
                    CallbackQueryHandler(button_HERBS)
                ],
                FIFTH : [ 
                    CallbackQueryHandler(button_STRATCH)
                ],
                
                SIXTH :[ 
                    CallbackQueryHandler(button_SWEET)
                ],
                SEVENTH : [ 
                    CallbackQueryHandler(end),

                ],
            },
            fallbacks=[CommandHandler('start', start)],
#             per_message=True
)

    updater.dispatcher.add_handler(conv_handler)
    updater.dispatcher.bot_data.update(survey_dict)
#     updater.dispatcher.add_handler(CallbackQueryHandler(button))
# #     updater.dispatcher.add_handler(MessageHandler( button2))
#     updater.dispatcher.add_handler(CommandHandler('help', help_command))

    # Start the Bot
    updater.start_polling()

    # Run the bot until the user presses Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT
    updater.idle()



In [7]:
def prep_type(answers_array):
    array =answers_array
#     for el in answers_array:
#         array.append(el[0])
    meat_prep ={ 0 : [0, 2, 3],
                 2 : [0, 1, 2, 3],
                 4 : [0, 1 , 2],
                 5 : [0, 1, 2, 4]
               }
    if array[0] in meat_prep.keys():
        array[1] = meat_prep[array[0]][array[1]]
    return array

In [8]:
def union2(dict1, dict2):
    dict_new = dict1
    for k,v in dict2.items():
        if (k in dict1.keys()) & (k in dict2.keys()):
            if k in dict_new.keys():
                dict_new[k] += v
            else:
                dict_new[k] = v
    return dict_new

In [9]:
def get_wine_type(answers_array):
    array_wines = np.array([
    'Bold Red',
    'Medium Red',
    'Light Red',
    'Rose',
    'Rich White',
    'Light White',
    'Sparkling',
    'Sweet White',
    'Dessert'
    ])
    
    wine_indexes = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
    survey_tensor = np.array([
    [ 
    [4, 1, 0, 0, 0, 0, 0, 0 ,0 ],
    [1, 1, 4, 1, 0, 0, 1, 2, 1 ],
    [1, 4, 0, 1, 0, 0, 1, 0, 0 ],
    [0, 1, 4, 1, 4, 1, 1, 0, 0 ],
    [0, 0, 0, 0, 0, 1, 4, 0, 0 ],
    [0, 0, 0, 0, 1, 4, 1, 0, 0 ],
    [0, 0, 0, 1, 4 ,1, 0, 1, 0 ],
#     [0, 0, 0, 0, 0, 0, 0, 0, 0 ]
],
     [
    [4, 1, 1, 0, 0, 0, 1, 1, 0],
    [0, 0, 4, 1, 1, 1, 1, 0, 0],
    [1, 4, 1, 1, 0, 0, 1, 0, 1],
    [4, 1, 1, 1, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1, 4, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0]
],   
    [
    [0, 1, 4, 1, 4, 1, 1, 1, 1],
    [1, 4, 0, 1, 0, 1, 1, 1, 4],
    [4, 1, 0, 1, 1, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0]
],
    [
    [1, 4, 1, 1, 1 ,1, 1, 1, 0],
    [0, 0, 0, 0, 0, 4, 1, 0, 0],
    [0, 0, 0, 4, 1, 0, 0, 1, 0],
    [1, 4, 0, 1, 0, 0, 0, 1, 0],
    [1, 4, 4, 0, 4, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 4, 0], 
    [0, 1, 0, 1, 0, 4, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0]
],
    [
    [4, 1, 0, 0, 0, 0, 0, 0, 0], 
    [1, 4, 0, 1, 0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 1, 1, 4, 0],
    [0, 1, 1, 1, 1, 4, 0, 0, 0],
    [0, 1, 0, 1, 0, 0, 0, 1, 4],
    [0, 4, 1, 1, 0, 0, 1, 4, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0]
],
    [
    [1, 1, 1 ,1, 1, 1, 1, 1, 1],
    [0, 0, 1 ,1, 1, 0, 0, 4, 0],
    [0, 0, 0, 1, 0, 0, 0, 4, 0],
    [1, 1, 1 ,1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0]
],
    [
    [0, 0, 0, 0, 0, 0, 1, 4, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 4],
    [0, 0, 0, 0, 0, 0, 0, 0, 0]
]
    ])
    
    
#     print(answers_array)
    array = prep_type(answers_array)
#     print(array)
    s = 0
    dict_dish_indexes = {}
    main_dish_indexes =  wine_indexes[np.where(np.array(survey_tensor[0][array[0]])!=0)[0]]
    print(main_dish_indexes)
#     print(survey_tensor[0][array[0]] ,np.where(np.array(survey_tensor[0][array[0]])!=0),main_dish_indexes)
    for i in range(len(main_dish_indexes)):
#         print(np.array(survey_tensor[1][array[1]])[np.where(survey_tensor[1][array[1]] !=0)])
#         print(np.array(survey_tensor[0][array[0]])[np.where(np.array(survey_tensor[0][array[0]])!=0)[0]])
        dict_dish_indexes[main_dish_indexes[i]] = np.array(survey_tensor[0][array[0]])[np.where(np.array(survey_tensor[0][array[0]])!=0)[0]][i]
    prep_type_indexes =  wine_indexes[np.where(np.array(survey_tensor[1][array[1]])!=0)]   
    for i in range(len(prep_type_indexes)):
        
        dict_dish_indexes[prep_type_indexes[i]] = +np.array(survey_tensor[1][array[1]])[np.where(np.array(survey_tensor[1][array[1]])!=0)[0]][i]
    print(dict_dish_indexes)    
    dict_rest_dishes = {}
    set_rest_whines = set()
    for i in range(2,len(array)):
        set_rest_whines.update(wine_indexes[np.where(np.array(survey_tensor[i][array[i]])!=0)[0]])
        print(wine_indexes[np.where(np.array(survey_tensor[i][array[i]]) !=0)])
        wi = wine_indexes[np.where(np.array(survey_tensor[i][array[i]])!=0)]
        print('wi', wi)
        print(np.array(survey_tensor[i][array[i]])[np.where(np.array(survey_tensor[i][array[i]])!=0)])
        if wi != []:
            print(800)
            for j in range(len(wi)):
#                 print('arar', np.array(survey_tensor[i][array[i]])[np.where(np.array(survey_tensor[i][array[i]])!=0)[0]])
                if  wi[j] in dict_rest_dishes.keys():
                    dict_rest_dishes[wi[j]] += np.array(survey_tensor[i][array[i]])[np.where(np.array(survey_tensor[i][array[i]])!=0)][j]
                else:
#                     print('ararrra', np.array(survey_tensor[i][array[i]])[np.where(np.array(survey_tensor[i][array[i]])!=0)[0]][j])
                    dict_rest_dishes[wi[j]] = np.array(survey_tensor[i][array[i]])[np.where(np.array(survey_tensor[i][array[i]])!=0)][j]
#         s +=np.array(survey_tensor[i][array[i]])
    if len(set_rest_whines)==0:
        first_phrase = np.array(["we could not find perfect match"])
        dict_dish_indexes_sorted = dict(sorted(dict_dish_indexes.items(), key=lambda item: -item[1]))
        answer = array_wines[np.array(list(dict_dish_indexes_sorted.keys()))]
        answer =np.concatenate((first_phrase, answer))
        
        
    else:
        set_main_dishes = set(list(dict_dish_indexes.keys()))
        if len(set_main_dishes & set_rest_whines) == 0:
            first_phrase = np.array(["we could not find perfect match"])
            dict_dish_indexes_sorted = dict(sorted(dict_dish_indexes.items(), key=lambda item: -item[1]))
            answer = array_wines[np.array(list(dict_dish_indexes_sorted.keys()))]
            answer =np.concatenate((first_phrase, answer))
        else:
            
            first_phrase = np.array(["perfect match for you"])
            print(dict_dish_indexes,  dict_rest_dishes)
            union_dict = union2(dict_dish_indexes, dict_rest_dishes)
            union_dict_sorted = dict(sorted(union_dict.items(), key=lambda item: -item[1]))
            answer = array_wines[np.array(list(union_dict.keys()))]
            answer =np.concatenate((first_phrase, answer))
        
#     print(s)
#     indx = np.argsort(-s)
#     print(indx)
#     answer =  array_wines[indx]
    return answer

In [10]:
if __name__ == '__main__':
    main()

/Users/Anna.Rudenko/opt/anaconda3/lib/python3.7/site-packages/telegram/ext/conversationhandler.py:288: UserWarning: If 'per_message=False', 'CallbackQueryHandler' will not be tracked for every message.
  "If 'per_message=False', 'CallbackQueryHandler' will not be "
2021-09-24 16:36:12,572 - apscheduler.scheduler - INFO - Scheduler started


Red meat
Cured meat 
Pork
Poiltry
Mollusk
Fish
Lobster & Shellfish
{121612247: {'questions': ('Red meat', 'Cured meat ', 'Pork', 'Poiltry', 'Mollusk', 'Fish', 'Lobster & Shellfish'), 'message_id': 401, 'chat_id': 121612247, 'answers': 0, 'count': 1, 'arr_q': ['q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7'], 'array_of_answers': [0, 0, 0, 0, 0, 0, 0]}}
{'message': {'channel_chat_created': False, 'entities': [], 'text': 'MEAT', 'chat': {'first_name': 'Natasha', 'id': 121612247, 'type': 'private', 'last_name': 'Palnik'}, 'reply_markup': {'inline_keyboard': [[{'text': 'Red meat', 'callback_data': '0'}], [{'text': 'Cured meat ', 'callback_data': '1'}], [{'text': 'Pork', 'callback_data': '2'}], [{'text': 'Poiltry', 'callback_data': '3'}], [{'text': 'Mollusk', 'callback_data': '4'}], [{'text': 'Fish', 'callback_data': '5'}], [{'text': 'Lobster & Shellfish', 'callback_data': '6'}]]}, 'message_id': 401, 'delete_chat_photo': False, 'caption_entities': [], 'photo': [], 'supergroup_chat_created': False, 

/Users/Anna.Rudenko/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:100: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
2021-09-24 17:48:38,437 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2021-09-24 17:48:38,438 - apscheduler.scheduler - INFO - Scheduler has been shut down


In [8]:
for i, el in enumerate(survey_dict['q1'][1]):
    print(i, el )

0 Red meat
1 Cured meat 
2 Pork
3 Poiltry
4 Mollusk
5 Fish
6 Lobster & Shellfish


In [11]:
aa= [3, 1, 0, 4, 3, 1, 0]

In [12]:
prep_type(aa)

IndexError: list index out of range

In [ ]:
def prep_type(answers_array):
    array = 
#     for el in answers_array:
#         array.append(el[0])
    meat_prep ={ 0 : [0, 2, 3],
                 2 : [0, 1, 2, 3],
                 4 : [0, 1 , 2],
                 5 : [0, 1, 2, 4]
               }
    if array[0] in meat_prep.keys():
        array[1] = meat_prep[array[0]][array[1]]
    return array